In [31]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random
from datetime import datetime, timedelta

def generate_random_dates():
    current_year = datetime.now().year
    start_date = datetime(current_year, 1, 1) + timedelta(days=random.randint(0, 364))
    end_date = start_date + timedelta(days=random.randint(1, 14))
        
    return  (start_date.month, start_date.day), (end_date.month, end_date.day)

with open(r'Usergrid Issues Summary 929.csv', newline='') as csvfile:
    csv_reader = csv.reader(csvfile)
    data = [row for row in csv_reader]

data.remove(data[0])

for pack in data:
    res = generate_random_dates()
    pack.append(res[0])
    pack.append(res[1])
    start_date = datetime(2024, res[0][0], res[0][1])
    end_date = datetime(2024, res[1][0], res[1][1])
    print((end_date - start_date).days)
print(data)

2
1
11
2
8
8
-359
7
4
11
10
6
13
13
3
7
8
3
8
4
5
1
1
2
-353
11
13
6
8
6
2
11
9
10
5
13
5
11
4
1
-353
1
1
14
2
10
8
1
9
8
11
14
10
11
13
11
7
11
-354
1
9
9
8
9
9
1
7
3
4
10
12
11
14
5
11
14
2
5
-362
-359
1
9
14
10
12
9
6
13
1
8
12
1
13
3
1
2
3
11
9
11
6
3
12
8
9
10
14
7
4
10
2
5
10
1
7
11
14
9
1
4
5
3
8
6
11
2
2
7
7
2
9
7
5
14
8
6
7
12
9
2
-358
1
9
-352
7
2
11
2
9
5
5
14
11
12
5
8
3
4
2
1
3
2
10
1
10
11
2
4
11
10
5
2
6
5
6
7
4
5
9
11
4
12
7
9
12
8
12
4
4
1
2
6
2
13
12
11
12
7
4
11
11
7
14
6
9
3
-354
9
12
1
3
2
7
3
5
4
4
10
14
8
9
-358
7
8
4
13
8
4
11
13
10
7
4
8
7
5
12
5
10
10
6
3
5
6
14
9
6
1
3
13
6
1
7
5
3
3
13
5
5
1
6
14
2
11
13
2
14
14
10
9
4
5
12
13
11
7
5
4
5
10
10
11
12
6
8
9
13
13
2
2
10
14
10
10
1
2
6
6
12
13
14
13
14
6
5
12
3
12
10
5
8
12
6
9
7
13
7
1
12
4
14
4
11
8
11
10
8
11
4
3
2
7
1
10
6
-354
6
1
12
5
3
8
8
14
2
6
8
3
7
4
13
3
7
5
8
7
10
6
10
14
3
10
14
9
11
10
5
10
4
5
9
7
13
6
12
6
4
2
12
7
3
2
7
9
2
14
2
2
7
8
12
4
7
12
8
7
7
7
12
2
10
13
3
-360
5
7
7
1
5
5
4
2
9
12
9


In [30]:
from catboost import CatBoostRegressor, Pool
import pandas as pd

def process_data(raw_data):
    data = []
    for row in raw_data:
        start_date = datetime(2024, row[-2][0], row[-2][1])
        end_date = datetime(2024, row[-1][0], row[-1][1])
        duration = (end_date - start_date).days
        
        processed_row = {
            'feature1': int(row[0]),
            'feature2': int(row[1]),
            'status': row[2],
            'feature4': int(row[3]),
            'task_id': row[4],
            'task_summary_length': len(row[5]),
            'priority': int(row[6]),
            'user_name': row[7],
            'initial_story_point': int(row[8]),
            'current_story_point': int(row[9]),
            'duration': duration
        }
        data.append(processed_row)
    return pd.DataFrame(data)

data_df = process_data(data)
print(data_df)
categorical_features = ['status', 'task_id', 'user_name']
target = 'duration'

     feature1  feature2                    status  feature4        task_id  \
0          23         1                 Completed  12760224   USERGRID-269   
1          23         1                 Completed  12760224   USERGRID-269   
2          23         1                 Completed  12761452   USERGRID-276   
3          23         1                 Completed  12761800   USERGRID-295   
4          23         1                 Completed  12761802   USERGRID-296   
..        ...       ...                       ...       ...            ...   
924        23       226  NotCompletedWithinSprint  12936220  USERGRID-1251   
925        23       226  NotCompletedWithinSprint  12938749  USERGRID-1260   
926        23       226   CompletedInNotherSprint  12777977   USERGRID-423   
927        23       226   CompletedInNotherSprint  12927353  USERGRID-1193   
928        23       226   CompletedInNotherSprint  12939564  USERGRID-1267   

     task_summary_length  priority user_name  initial_story_poi

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Split features and target
X = data_df.drop(columns=[target])
y = data_df[target]

# Train-test split (assuming larger dataset in reality)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CatBoost Regressor
model = CatBoostRegressor(
    iterations=5000, 
    learning_rate=0.1, 
    depth=6, 
    cat_features=categorical_features, 
    verbose=10,
    task_type="GPU",
    devices='0'  
)

# Train the model
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

In [ ]:
# Predict duration for new task
new_task = pd.DataFrame([{
    'feature1': 25,
    'feature2': 2,
    'status': 'Completed',
    'feature4': 13829292,
    'task_id': 'NEW-TASK-101',
    'task_summary_length': 40,
    'priority': 5,
    'user_name': 'rbridges',
    'initial_story_point': 0,
    'current_story_point': 5
}])

predicted_duration = model.predict(new_task)
print(f"Predicted Duration: {predicted_duration} days")